<a href="https://colab.research.google.com/github/vjuge/oc-iml/blob/P6/modules/P6/module_P6_V3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import tensorflow as tf
import tensorflow_datasets as tfds
from keras.utils.vis_utils import plot_model
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageShow
import cv2 as cv

import tensorflow as tf
from tensorflow import keras
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.vis_utils import plot_model
from keras import backend as K
from keras.models import load_model
from tensorflow.keras import layers

from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


In [5]:
device_name = tf.test.gpu_device_name()
if "GPU" not in device_name:
    print("GPU device not found")
else: print('Found GPU at: {}'.format(device_name))

GPU device not found


## Download Dataset

In [16]:
# tfds.disable_progress_bar()

# train_ds, validation_ds, test_ds = tfds.load(
#     "stanford_dogs",
#     # Reserve 10% for validation and 10% for test
#     split=["train[:40%]", "train[40%:50%]", "train[50%:60%]"],
#     as_supervised=True
# )

train_ds, test_ds = tfds.load(
    "stanford_dogs",
    split=['train', 'test'],
    as_supervised=True
)

In [ ]:
# one-hot encode labels ?
# NUM_CLASSES=120
# train_ds = train_ds.map(lambda x, y: (x, tf.one_hot(y, depth=NUM_CLASSES)))
# test_ds = test_ds.map(lambda x, y: (x, tf.one_hot(y, depth=NUM_CLASSES)))

In [ ]:
import numpy as np

for images, labels in train_ds.take(1):
    plt.figure(figsize=(10, 10))
    first_image = images[0]
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        augmented_image = data_augmentation(
            tf.expand_dims(first_image, 0), training=True
        )
        plt.imshow(augmented_image[0].numpy().astype("int32"))
        plt.title(int(labels[0]))
        plt.axis("off")

Prepare the data : 

In [17]:
size = (150, 150)

train_ds = train_ds\
    .map(lambda x, y: (tf.image.resize(x, size), y))\
        .map(lambda x, y: (x, tf.one_hot(y, depth=120)))
# validation_ds = validation_ds.map(lambda x, y: (tf.image.resize(x, size), y))
test_ds = test_ds.map(lambda x, y: (tf.image.resize(x, size), y))\
    .map(lambda x, y: (x, tf.one_hot(y, depth=120))) #depth is number of classes

In [18]:
batch_size = 32

train_ds = train_ds.cache().batch(batch_size).prefetch(buffer_size=10)
#validation_ds = validation_ds.cache().batch(batch_size).prefetch(buffer_size=10)
test_ds = test_ds.cache().batch(batch_size).prefetch(buffer_size=10)

In [19]:
from tensorflow import keras
from tensorflow.keras import layers

data_augmentation = keras.Sequential(
    [layers.RandomFlip("horizontal"), layers.RandomRotation(0.1),]
)

Build model

In [20]:
base_model = keras.applications.Xception(
    weights="imagenet",  # Load weights pre-trained on ImageNet.
    input_shape=(150, 150, 3),
    include_top=False,
)  # Do not include the ImageNet classifier at the top.

# Freeze the base_model
base_model.trainable = False

# Create new model on top
inputs = keras.Input(shape=(150, 150, 3))
x = data_augmentation(inputs)  # Apply random data augmentation

# Pre-trained Xception weights requires that input be scaled
# from (0, 255) to a range of (-1., +1.), the rescaling layer
# outputs: `(inputs * scale) + offset`
scale_layer = keras.layers.Rescaling(scale=1 / 127.5, offset=-1)
x = scale_layer(x)

# The base model contains batchnorm layers. We want to keep them in inference mode
# when we unfreeze the base model for fine-tuning, so we make sure that the
# base_model is running in inference mode here.
x = base_model(x, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dropout(0.2)(x)  # Regularize with dropout
outputs = keras.layers.Dense(120, activation='softmax')(x)  #120 = number of breeds
model = keras.Model(inputs, outputs)

model.summary()

83697664/83683744 [==============================] - 2s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 sequential_1 (Sequential)   (None, 150, 150, 3)       0         
                                                                 
 rescaling (Rescaling)       (None, 150, 150, 3)       0         
                                                                 
 xception (Functional)       (None, 5, 5, 2048)        20861480  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dropout (Dropout)           (None, 2048)              0      

Train top layer

In [22]:
model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.CategoricalCrossentropy(),
    metrics=[keras.metrics.CategoricalAccuracy()],
)

epochs = 20
history = model.fit(train_ds, epochs=epochs, validation_data=test_ds)

Epoch 1/20
375/375 [==============================] - 178s 470ms/step - loss: 2.1303 - categorical_accuracy: 0.4663 - val_loss: 1.3805 - val_categorical_accuracy: 0.6234
Epoch 2/20
375/375 [==============================] - 177s 473ms/step - loss: 1.5406 - categorical_accuracy: 0.5841 - val_loss: 1.3786 - val_categorical_accuracy: 0.6350
Epoch 3/20
375/375 [==============================] - 181s 483ms/step - loss: 1.3702 - categorical_accuracy: 0.6229 - val_loss: 1.4129 - val_categorical_accuracy: 0.6364
Epoch 4/20
375/375 [==============================] - 182s 486ms/step - loss: 1.2902 - categorical_accuracy: 0.6332 - val_loss: 1.4123 - val_categorical_accuracy: 0.6340
Epoch 5/20
375/375 [==============================] - 188s 502ms/step - loss: 1.1807 - categorical_accuracy: 0.6653 - val_loss: 1.4700 - val_categorical_accuracy: 0.6334
Epoch 6/20
375/375 [==============================] - 185s 495ms/step - loss: 1.1293 - categorical_accuracy: 0.6733 - val_loss: 1.4760 - val_categoric

KeyboardInterrupt: 

probleme : est ce que les labels sont encodés en int ?

--> faire un label encoder avant ?

In [ ]:
for images, labels in train_ds.take(10):
    print(labels[0])

tf.Tensor(36, shape=(), dtype=int64)
tf.Tensor(99, shape=(), dtype=int64)
tf.Tensor(55, shape=(), dtype=int64)
tf.Tensor(41, shape=(), dtype=int64)
tf.Tensor(85, shape=(), dtype=int64)
tf.Tensor(79, shape=(), dtype=int64)
tf.Tensor(46, shape=(), dtype=int64)
tf.Tensor(83, shape=(), dtype=int64)
tf.Tensor(95, shape=(), dtype=int64)
tf.Tensor(36, shape=(), dtype=int64)


Data augmentation:

<https://pyimagesearch.com/2019/07/08/keras-imagedatagenerator-and-data-augmentation/>


### model compile

What are the choices for optimizer, loss & metrics ? : <https://keras.io/guides/training_with_built_in_methods#many-builtin-optimizers-losses-and-metrics-are-available>

* optimizers: <https://keras.io/api/optimizers/>
* Losses: <https://keras.io/api/losses/>
* Metrics: <https://keras.io/api/metrics/>



tip: in order to view GPU usage, type command: `watch -n 1 nvidia-smi`

The results are quite poor, both accuracy and loss are too low (resp. too high)

## Pre-trained model : Transfer Learning

Kera doc: <https://keras.io/guides/transfer_learning/>

deep learning with Python <https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html>

